# Text-mined dataset

In [2]:
!python ../../s4/scripts/001.thermonize_dmm.py \
    --download-fn intermediate_data/Reactions_Solid_State_legacy_v14_2.pypickle \
    --mp-output intermediate_data/DMM_Thermonized_MP.pypickle \
    --freed-output intermediate_data/DMM_Thermonized_FREED.pypickle

The pickle file intermediate_data/Reactions_Solid_State_legacy_v14_2.pickle does not exist, try to download from database...
100%|███████████████████████████████████| 30004/30004 [00:14<00:00, 2112.98it/s]
Downloaded 30004 reactions from SynPro
Found 30004 Reactions
100%|███████████████████████████████████| 30004/30004 [1:53:38<00:00,  4.40it/s]
Thermonized 2 reactions using MP data.
100%|██████████████████████████████████| 30004/30004 [00:02<00:00, 13880.57it/s]
Thermonized 2 reactions using FREED data.


In [17]:
import pickle
from collections import Counter

with open('intermediate_data/DMM_Thermonized_MP.pypickle', 'rb') as f:
    data = pickle.load(f)

for k in data[1]:
    v = data[1][k]
    error_counting = Counter(x['exception_desc'] for x in v)

    print('='*70)
    print('Exception', k)

    for err, cnt in error_counting.items():
        print('%04d %s' % (cnt, err))

Exception <class 'ValueError'>
5111 Linear system has no solution
0020 This material has 11 thermo entries, please specify you need a list by setting allow_many=True.
0008 could not convert string to float: '1-'
0008 could not convert string to float: '6.0-'
0037 could not convert string to float: '12.0-'
0062 could not convert string to float: '3-'
0023 could not convert string to float: '2.0-4'
0001 Can't parse Element or String from type <class 'str'>: Nx.
0012 could not convert string to float: '2-'
0002 could not convert string to float: '2.0-'
0002 could not convert string to float: '4.0-'
0017 could not convert string to float: '3-2'
0003 could not convert string to float: '.'
0003 could not convert string to float: '4-'
0002 could not convert string to float: '0.9-'
0001 could not convert string to float: '1.0-4'
0007 could not convert string to float: '3-3'
0001 could not convert string to float: '3-4'
0001 could not convert string to float: '2.0-2'
0001 could not convert stri

In [38]:
!python ../../s4/scripts/002.compute_cascades.py \
    --reactions-file intermediate_data/DMM_Thermonized_MP.pypickle \
    --output-fn intermediate_data/DMM_Cascades.pypickle \
    --cascade-temperatures '[500,750,1000,1250,1500,1750]'

Loading dataset from intermediate_data/DMM_Thermonized_MP.pypickle
Found 21039 recipes
Found 16070 valid recipes with temperature above 500 degC
Computing cascades at the specified temperatures: [500, 750, 1000, 1250, 1500, 1750]...
Cascades: 100%|█████████████████████████| 16070/16070 [4:45:32<00:00,  1.07s/it]


In [62]:
import pickle
from collections import defaultdict, Counter

with open('intermediate_data/DMM_Cascades.pypickle', 'rb') as f:
    data = pickle.load(f)
    
cascade_error = defaultdict(Counter)

for key in data:
    for k, reaction in data[key]:
        for temp, cascade in reaction.items():
            if cascade['error']:
                cascade_error[cascade['exception']][cascade['exception_desc']] += 1
                q = cascade
for exp_name in cascade_error:
    print('='*70)
    print('Exception', exp_name)

    for err, cnt in cascade_error[exp_name].items():
        print('%04d %s' % (cnt, err))

Exception <class 's4.cascade.balance.CannotBalance'>
0217 Target is included in precursor/open_comp list.
Exception <class 'TypeError'>
0328 '<' not supported between instances of 'NoneType' and 'NoneType'
0006 '<' not supported between instances of 'NoneType' and 'float'


In [78]:
!python ../../s4/scripts/003.prepare_ml_data.py \
    --reactions-file intermediate_data/DMM_Thermonized_MP.pypickle \
    --text-mined-recipes-file intermediate_data/Reactions_Solid_State_legacy_v14_2.pypickle \
    --output-fn intermediate_data/DMM_TrainingData.pypickle \
    --cascade-file intermediate_data/DMM_Cascades.pypickle

Loaded 21039 reaction data
Loaded 30004 TMS data
All MP
Analysis of cascade results (per recipe):
Successful: 5999, failed: 11566, joint (has both success/failure): 1524
Featurizing: 100%|██████████████████████████| 7562/7562 [45:01<00:00,  2.80it/s]
Computed 7562 entries
Writing results to intermediate_data/DMM_TrainingData.pypickle


# PCD dataset

In [15]:
with open('intermediate_data/PCD_Reactions.pypickle', 'wb') as f:
    pickle.dump((data, {}), f)

In [16]:
!python ../../s4/scripts/002.compute_cascades.py \
    --reactions-file intermediate_data/PCD_Reactions.pypickle \
    --output-fn intermediate_data/PCD_Cascades.pypickle \
    --cascade-temperatures '[1000]'

Loading dataset from intermediate_data/PCD_Reactions.pypickle
Found 32960 recipes
Found 32166 valid recipes with temperature above 500 degC
Computing cascades at the specified temperatures: [1000]...
Cascades: 100%|███████████████████████████| 32166/32166 [29:55<00:00, 17.91it/s]


In [17]:
import pickle
from collections import defaultdict, Counter

with open('intermediate_data/PCD_Cascades.pypickle', 'rb') as f:
    data = pickle.load(f)
    
cascade_error = defaultdict(Counter)

for key in data:
    for k, reaction in data[key]:
        for temp, cascade in reaction.items():
            if cascade['error']:
                cascade_error[cascade['exception']][cascade['exception_desc']] += 1
                q = cascade
for exp_name in cascade_error:
    print('='*70)
    print('Exception', exp_name)

    for err, cnt in cascade_error[exp_name].items():
        print('%04d %s' % (cnt, err))

Exception <class 'TypeError'>
0008 '<' not supported between instances of 'NoneType' and 'float'
Exception <class 'KeyError'>
0008 'Pa'
Exception <class 'ValueError'>
0004 Linear system has no solution
Exception <class 's4.cascade.balance.CannotBalance'>
0004 Target is included in precursor/open_comp list.


In [18]:
!python ../../s4/scripts/003.prepare_ml_data.py \
    --reactions-file intermediate_data/PCD_Reactions.pypickle \
    --output-fn intermediate_data/PCD_TrainingData.pypickle \
    --cascade-file intermediate_data/PCD_Cascades.pypickle

Loaded 32960 reaction data
All MP
Analysis of cascade results (per recipe):
Successful: 7677, failed: 24477, joint (has both success/failure): 0
Featurizing:   9%|██▎                        | 672/7677 [02:10<52:37,  2.22it/s]/home/hhuo/anaconda3/envs/synthesis/lib/python3.7/site-packages/pymatgen/ext/matproj.py:586: DeprecationWarning: __init__ is deprecated
MaterialsProjectCompatibility will be updated with new correction classes as well as new values of corrections and uncertainties in 2020
  entries = MaterialsProjectCompatibility().process_entries(entries)
Featurizing:  37%|█████████▋                | 2846/7677 [09:43<15:25,  5.22it/s]/home/hhuo/anaconda3/envs/synthesis/lib/python3.7/site-packages/pymatgen/ext/matproj.py:586: DeprecationWarning: __init__ is deprecated
MaterialsProjectCompatibility will be updated with new correction classes as well as new values of corrections and uncertainties in 2020
  entries = MaterialsProjectCompatibility().process_entries(entries)
Featurizing